# Gambit Experiment

This notebook experiments with the Gambit software package to understand its capabilities and how it might fit with our efforts for monte carlo simulations and explainability

In [1]:
import pygambit as gbt
import numpy as np


### Game Setup
Building an extensive game. Could be useful for future use when there are multi stage games.

In [2]:
g = gbt.Game.new_tree(players=["Alice", "Bob"],
                      title="One card poker game, after Myerson (1991)")

In [3]:
g.append_move(g.root, g.players.chance, ["King", "Queen"])
for node in g.root.children:
    g.append_move(node, "Alice", ["Raise", "Fold"])
g.append_move(g.root.children[0].children[0], "Bob", ["Meet", "Pass"])
g.append_infoset(g.root.children[1].children[0],
                 g.root.children[0].children[0].infoset)
alice_winsbig = g.add_outcome([2, -2], label="Alice wins big")
alice_wins = g.add_outcome([1, -1], label="Alice wins")
bob_winsbig = g.add_outcome([-2, 2], label="Bob wins big")
bob_wins = g.add_outcome([-1, 1], label="Bob wins")
g.set_outcome(g.root.children[0].children[0].children[0], alice_winsbig)
g.set_outcome(g.root.children[0].children[0].children[1], alice_wins)
g.set_outcome(g.root.children[0].children[1], bob_wins)
g.set_outcome(g.root.children[1].children[0].children[0], bob_winsbig)
g.set_outcome(g.root.children[1].children[0].children[1], alice_wins)
g.set_outcome(g.root.children[1].children[1], bob_wins)

Build a normal form game

In [6]:
m = np.array([[8, 2], [10, 5]])
t = gbt.Game.from_arrays(m, np.transpose(m))

In [7]:
t.outcomes

GameOutcomes(game=Game(title='Untitled strategic game'))

### Computing Nash Equilibria

With Extensive Games - not all fuctionality explored

In [8]:
result = gbt.nash.lcp_solve(g)
eqm = result.equilibria[0]
eqm['Alice']

[[Rational(1, 1), Rational(0, 1)], [Rational(1, 3), Rational(2, 3)]]

In [9]:
eqm["Bob"]

[[Rational(2, 3), Rational(1, 3)]]

Solving Prisoners Dilemma with Pure Strategy

In [4]:
def create_prisoners_dilemma(payoff1, payoff2):
    """
    Create a Prisoner's Dilemma game using Pygambit.
    
    Parameters:
    - cooperate_reward: Payoff when both players cooperate (default: -1)
    - defect_reward: Payoff when both players defect (default: 0)
    - temptation: Payoff for defecting while other cooperates (default: 1)
    - sucker: Payoff for cooperating while other defects (default: -2)
    
    Returns:
    - game: Pygambit strategic game object
    """
    # Create a new strategic game with 2 players, 2 strategies each
    game = gbt.Game.from_arrays(payoff1, payoff2)
    
    # Set the game title
    game.title = "Prisoner's Dilemma"
    
    # Set player names
    game.players[0].label = "Prisoner 1"
    game.players[1].label = "Prisoner 2"
    
    # Set strategy labels for both players
    game.players[0].strategies[0].label = "Cooperate"
    game.players[0].strategies[1].label = "Defect"
    game.players[1].strategies[0].label = "Cooperate"
    game.players[1].strategies[1].label = "Defect"
    
    return game

In [5]:
payoff_player1 = [[1,2],[3,4]]
payoff_player2 = [[5,6],[7,8]]
g = create_prisoners_dilemma(payoff_player1, payoff_player2)

In [6]:
results = gbt.nash.enumpure_solve(g, use_strategic = True) 

In [7]:
results.game

,Cooperate,Defect
Cooperate,"1,5","2,6"
Defect,"3,7","4,8"


In [8]:
results.equilibria[0]['Prisoner 1']

[Rational(0, 1), Rational(1, 1)]

In [9]:
float(results.equilibria[0]['Prisoner 1']['Cooperate']*100)

0.0

In [10]:
payoff_player1 = [[-1,-10],[0,-6]]
payoff_player2 = np.transpose(payoff_player1)
g = create_prisoners_dilemma(payoff_player1, payoff_player2)

In [11]:
results = gbt.nash.enumpure_solve(g) 

In [12]:
results.game

,Cooperate,Defect
Cooperate,"-1,-1","-10,0"
Defect,"0,-10","-6,-6"


In [13]:
results.equilibria[0]['Prisoner 2']

[Rational(0, 1), Rational(1, 1)]

In [16]:
float(results.equilibria[0]['Prisoner 1']['Cooperate']*100)

0.0

In [15]:
# To note: Player 1 on the rows, Player 2 on the columns 